<div class="alert alert-block alert-info">

## <center> GROUP PROJECT - TO GRANT OR NOT TO GRANT: DECIDING ON COMPENSATION BENEFITS </center> <br>
#  <center> <b> PREPROCESS </center> <br>
## <center> Fall Semester 2024-2025 <center>
<br>
<center> Group 46: <center> <br>
<center>Afonso Ascensão, 20240684 <br><center>
<center>Duarte Marques, 20240522 <br><center>
<center>Joana Esteves, 20240746 <br><center>
<center>Rita Serra, 20240515 <br><center>
<center>Rodrigo Luís, 20240742 <br><center>

<div>

**Description of notebook contents:**
- Apply tranformations that could be implemented before the split, namely: dropping variables selected during EDA for being problematic, correction of data types, correction of incongruencies and part of the feature engineering.
- Perform a split that will be used for all model's notebooks. Generate files for train and validation data before and after split to use as needed for the models.
- Implement a pipeline to do after split tranformations inlcluding: outlier handeling, missing values inputation, feature engineering, categoric variables cardinality reduction, encoding, scaling and performing feature selection.

**Table of Contents**
- [1. Import the needed Libraries](#importlibraries)
- [2. Import Dataset](#importdataset)
- [3. Preprocessing](#section_3)
    - [3.1. Variables to drop](#section_3_1)
    - [3.2. Data Types](#section_3_2)
    - [3.3. Strange Values](#section_3_3)
    - [3.4. Feature Engineering](#section_3_4)
    - [3.5. Split](#section_3_5)
    - [3.6. Pipeline](#section_3_6)


<a class="anchor" id="importlibraries">

# 1. Import the needed Libraries

</a>

In [25]:
#%pip install pyarrow
#%pip install joblib

import pandas as pd
import numpy as np

# Split
from sklearn.model_selection import train_test_split

# Pipeline
from sklearn.pipeline import Pipeline
## Export pipeline
import joblib
## py file with custom tranformer for pipeline
from transformers import *

# Display all rows 
pd.set_option('display.max_rows', None)

<a class="anchor" id="importdataset">

# 2. Import Dataset

</a>

In [3]:
train_data = pd.read_csv('train_data.csv', sep = ',')
test_data = pd.read_csv('test_data.csv', sep = ',')

C:\Users\rcouc\AppData\Local\Temp\ipykernel_5048\1422952055.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('train_data.csv', sep = ',')


<a class="anchor" id="section_3">

# 3. Preprocessing

</a>

<a class="anchor" id="section_3_1">

## 3.1. Variables to drop

</a>

According to our EDA this variables have a high percentage of nan and are redudant with other variables, so we will drop them.

In [4]:
# Variable with 100% nan
train_data.drop(["OIICS Nature of Injury Description"], axis=1, inplace=True)
test_data.drop(["OIICS Nature of Injury Description"], axis=1, inplace=True)

In [5]:
# This variable is 5% nan and is redundant with "Medical Fee Region" (Cramer's V Matrix shows correlation of 1), 
# and highly correlated with other location variables.
train_data.drop(["Zip Code"], axis=1, inplace=True)
test_data.drop(["Zip Code"], axis=1, inplace=True)

In [6]:
# Correspondent codes and descriptions have a correlation of 1 and as such the information is redundant
train_data = train_data.drop(columns=['WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description','Industry Code Description'])
test_data = test_data.drop(columns=['WCIO Cause of Injury Description','WCIO Nature of Injury Description','WCIO Part Of Body Description','Industry Code Description'])

Note: Birth Year is redudant with age at injury and is 9% nan, we will keep it to input nan values and after that we will remove it.

We will also drop the variables in the train dataset that won't be used as features:

In [7]:
train_data.drop(['WCB Decision'], axis=1, inplace=True)

<a class="anchor" id="section_3_2">

## 3.2. Data Types

</a>

In [8]:
# Convert variables related to codes to strings
code_columns = ["Industry Code", "WCIO Cause of Injury Code",
                "WCIO Nature of Injury Code", "WCIO Part Of Body Code"]

train_data[code_columns] = train_data[code_columns].applymap(lambda x: str(x) if pd.notna(x) else x)

test_data[code_columns] = test_data[code_columns].applymap(lambda x: str(x) if pd.notna(x) else x)

C:\Users\rcouc\AppData\Local\Temp\ipykernel_5048\1867619682.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_data[code_columns] = train_data[code_columns].applymap(lambda x: str(x) if pd.notna(x) else x)
C:\Users\rcouc\AppData\Local\Temp\ipykernel_5048\1867619682.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_data[code_columns] = test_data[code_columns].applymap(lambda x: str(x) if pd.notna(x) else x)


In [9]:
#Convert some columns to date format as it's more suitable

date_columns = ['Accident Date', 'C-2 Date', 'C-3 Date', 'Assembly Date', 'First Hearing Date']

for column in date_columns:
    train_data[column] = pd.to_datetime(train_data[column], errors='coerce')
    test_data[column] = pd.to_datetime(test_data[column], errors='coerce')

In [10]:
# Convert some columns to integer format as it's more suitable
# Use int64 to keep nan for now

int_columns = ['Age at Injury', 'IME-4 Count', 'Number of Dependents']

for column in int_columns:
    train_data[column] = train_data[column].astype('Int64')
    test_data[column] = test_data[column].astype('Int64')

In [11]:
# Convert "Attorney/Representative" to boolean
train_data["Attorney/Representative"] = train_data["Attorney/Representative"].map({"Y": True, "N": False}).astype(bool)
test_data["Attorney/Representative"] = test_data["Attorney/Representative"].map({"Y": True, "N": False}).astype(bool)

# Convert "COVID-19 Indicator" to boolean
train_data["COVID-19 Indicator"] = train_data["COVID-19 Indicator"].map({"Y": True, "N": False}).astype(bool)
test_data["COVID-19 Indicator"] = test_data["COVID-19 Indicator"].map({"Y": True, "N": False}).astype(bool)

<a class="anchor" id="section_3_3">

## 3.3. Strange values

</a>

From our EDA we concluded:
- There are invalid dates, where Assemby Date is earlier than Accident Date, we will correct this cases.
- All 9 digit claim id lines have missing values for all varibles except "Assembly Date", so we will remove this lines.
- Numeric features: There are zero values that can be interpreted as missing values and need imputation - namely Age at injury, Average weekly wage and Birth year.
- Categoric features: There are some variables we believe should be binary and will consider categories like U (unkown) or X (possibly non-binary - very low frequency) as missing values for future inputation.

In [12]:
# For the detected cases change accident date to match assembly date
train_data.loc[(train_data['Assembly Date'] < train_data['Accident Date']), "Accident Date"] = train_data['Assembly Date']
test_data.loc[(test_data['Assembly Date'] < test_data['Accident Date']), "Accident Date"] = test_data['Assembly Date']

In [13]:
# Select all lines with a 9 digit clain identifier
nine_digit_id = train_data.loc[train_data["Claim Identifier"] > 10_000_000]

# Remove 9 digit claim id lines from the dataset
train_data = train_data.drop(nine_digit_id.index)

In [14]:
# Replacing the 0 values with nan so it's easier to correct them 

train_data['Average Weekly Wage'] = train_data['Average Weekly Wage'].replace(0, np.nan)
test_data['Average Weekly Wage'] = test_data['Average Weekly Wage'].replace(0, np.nan)

train_data['Birth Year'] = train_data['Birth Year'].replace(0, np.nan)
test_data['Birth Year'] = test_data['Birth Year'].replace(0, np.nan)

train_data['Age at Injury'] = train_data['Age at Injury'].replace(0, np.nan)
test_data['Age at Injury'] = test_data['Age at Injury'].replace(0, np.nan)

In [15]:
# Replacing U and X categories with nan so it's easier to correct them

train_data['Gender'] = train_data['Gender'].replace(["U", "X"], np.nan).map({'M': True, 'F': False}).astype(bool)
test_data['Gender'] = test_data['Gender'].replace(["U", "X"], np.nan).map({'M': True, 'F': False}).astype(bool)

train_data['Alternative Dispute Resolution'] = train_data['Alternative Dispute Resolution'].replace("U", np.nan).map({"Y": True, "N": False}).astype(bool)
test_data['Alternative Dispute Resolution'] = test_data['Alternative Dispute Resolution'].replace("U", np.nan).map({"Y": True, "N": False}).astype(bool)

<a class="anchor" id="section_3_4">

## 3.4. Feature Engineering

</a>

**Binary variables:**
- "Hearing Held": Indicates if an hearing was held or not.
- "C-2 Delivered": Indicates if this form was ever delivered. 
- "C-2 Delivered on Time": Indicates in the C-2 form was delivered in the established time (up to 10 days).
- "C-3 Delivered": Indicates if this form as ever delivered.
- "C-3 Delivered on Time" : Indicates if the form was delivered in the established time (up to 730 days).

**Mathematical tranformations:**
- "Average Weekly Wage Log": After zero removal (for the plot) we saw that Average Weekly Wage was right skewed, we will create a new feature with a log tranformation. 
- "IME-4 Count log": IME-4 Count is right skewed, we will create a new feature with a log transformation.

**Temporal lags:**
- "Time Accident to Assembly": Days between accident date and assembly date.
- "Time Assembly to Hearing": Days between assembly date and first hearing date.

**Dates to Year:**
- "Accident Year": There year when the accident happend. 
- "Assembly Year": The year when the claim was assembled.

**Union of binary variables:**
- 'At/rp OR altr dispute': Logical OR for Alternative Dispute Resolution and Attorney/Representative.
- 'Covid OR altr dispute': Logical OR for Covid 19 Indicator and Attorney/Representative.
- 'Covid OR At/rp': Logical OR for Covid 19 Indicator and Attorney/Representative.

**Ratios:**
- 'Wage Age Ratio': Ratio between the average weekly wage (after log) and the age.

**Extreme outlier flags:**
- 'High Wage Flag': Flags values in top 5% for the average weekly wage.
- 'High count IME-4 Flag': Flags values in top 1% for the number of IME-4 forms delivered.

In [16]:
# Dates to Year:

train_data['Accident Year'] = train_data['Accident Date'].dt.year
test_data['Accident Year'] = test_data['Accident Date'].dt.year

train_data['Assembly Year'] = train_data['Assembly Date'].dt.year
test_data['Assembly Year'] = test_data['Assembly Date'].dt.year

- The remaining feature will be computed in the pipeline after outliers and missing values handeling.

<a class="anchor" id="section_3_5">

## 3.5. Split

</a>

In [17]:
train_data = train_data.set_index("Claim Identifier")
test_data = test_data.set_index("Claim Identifier")

In [18]:
# Save dataset with before split tranformations (tranformations from 3.1 to 3.4) to use for CV in the models notebooks,
# where we will use the pipeline to complete the preprocessing

# Use parquet to perserve data types
train_data.to_parquet("transformed_train_data.parquet", index=True)
test_data.to_parquet("transformed_test_data.parquet", index=True)

In [19]:
X = train_data.drop(['Claim Injury Type'], axis = 1)
y = train_data['Claim Injury Type']

In [20]:
# Stratified split to deal with classe unbalance in the target
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.1,
                                                  random_state = 0,
                                                  stratify = y,
                                                  shuffle = True)

In [21]:
# Save train and validation data after split with before split tranformation to use for the models notebooks,
# where we will apply the pipeline to complete preprocessing
transformed_train_split = pd.concat([pd.DataFrame(X_train), pd.Series(y_train, name="Claim Injury Type")], axis=1)
transformed_val_split = pd.concat([pd.DataFrame(X_val), pd.Series(y_val, name="Claim Injury Type")], axis=1)

transformed_train_split.to_parquet("transformed_train_split.parquet", index=True)
transformed_val_split.to_parquet("transformed_val_split.parquet", index=True)

<a class="anchor" id="section_3_6">

## 3.6. Pipeline

</a>

In [27]:
# Preprocessing pipeline
pipeline = Pipeline([   
    ('impute', ImputeTransformer()),  # Imputation of missing values
    ('outliers', OutlierHandlingTransformer()),  # Outlier handling
    ('feature_engineering', FeatureEngineeringTransformer()), # Performs feature engineering 
    ('cardinality', HighCardinalityTransformer()), # Reduce classes of high cardinality features
    ('encoding', EncodingTransformer()),  # Encoding categorical variables
    ('scaling', ScalingTransformer()),  # Scaling numeric features
    ('feature_selection', FeatureSelectionTransformer()) # Selection features for models 
])

# Save pipeline
joblib.dump(pipeline, 'pipeline.joblib')

['pipeline.joblib']

In [23]:
''' # Testing 
X_train.drop(['Agreement Reached'], axis=1, inplace=True)
X_val.drop(['Agreement Reached'], axis=1, inplace=True)

X_train_preprocessed = pipeline.fit_transform(X_train)
X_val_preprocessed = pipeline.transform(X_val)
test_data_preprocessed = pipeline.transform(test_data) 
'''

" # Testing \nX_train.drop(['Agreement Reached'], axis=1, inplace=True)\nX_val.drop(['Agreement Reached'], axis=1, inplace=True)\n\nX_train_preprocessed = pipeline.fit_transform(X_train)\nX_val_preprocessed = pipeline.transform(X_val)\ntest_data_preprocessed = pipeline.transform(test_data) \n"